# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

In [ ]:
#Load the csv exported in Part I to a DataFrame
file_to_load = "../output_data/weather.csv"
city_data = pd.read_csv(file_to_load)
city_data.head()

In [ ]:
#city_data.shape

In [ ]:
city_df = city_data.dropna()
city_df.head()

In [ ]:
#city_df.shape

In [ ]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
#Use the Lat and Lng as locations
location_df = city_df[["Latitude", "Longitude"]].astype(float)

# Use humidity as the weight
humidity_df = city_df["Humidity"].astype(float)

In [ ]:
#Add Heatmap layer to map
fig = gmaps.figure(center =[0,0], zoom_level = 3)

heat_layer = gmaps.heatmap_layer(location_df, weights = humidity_df, 
                                 dissipating = False, 
                                 max_intensity = 100, 
                                 point_radius = 1 )
fig.add_layer(heat_layer)
fig

In [ ]:
vocation_cities = pd.DataFrame()

for index, row in city_df.iterrows():
    if row["Max Temp"]>70 and row["Max Temp"]<80 and row["Wind Speed"]< 10 and row["Cloudiness"]==0:
        vocation_cities[index] = row
        
clean_df = vocation_cities.T

In [ ]:
del clean_df["Unnamed: 0"]

In [ ]:
reset_df = clean_df.reset_index()

In [ ]:
reset_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Store into variable named hotel_df
hotel_df =reset_df
hotel_df

In [ ]:
#add a new column as "Hotel Name"to the current dataframe     
hotel_df["Hotel Name"] = ""

params = {"radius": 50000, "types": "hotel", "keyword": "hotels", "key": g_key}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
  
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # not to crush the code if no data found
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#hotel_df      
        

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
fig =gmaps.figure()
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)
# Display figure
fig